In [ ]:
from packages import Exam
from packages import Unit
from packages.Unit import N, m, m2, m3, m4, pers, Pa,  K, Nm, Nm2, non
import numpy as np

path = ["./files/result/csv/top_sample.csv",
        "./files/result/csv/back_sample.csv"]
unit_instances = [N(), m(), m2(), m3(), m4(), pers(), Pa(), K()]

exam = Exam.Exam(path)
exam.get_marks()
exam.get_student()
#q1
exam.parts.append(Exam.SingleAlphabetPart(marks=exam.marks[0][4:9, 1:27], corrects=["A","Q","B","H","P"], allocations=[2] * 5))
#q2
exam.parts.append(Exam.SingleAlphabetPart(marks=exam.marks[0][10:20, 1:27], corrects=["A","E","B","C","H","I","J","M","N","O"], allocations=[2] * 10))
#q3
exam.parts.append(Exam.DualNumberPart(
    marks = np.vstack([exam.marks[0][21:37, 1:27], exam.marks[0][2:6, 29:55]]),
    unit_instances=unit_instances,
    corrects = [
        [{"value":1e3, "unit":N()}, {"value":-3e3, "unit":Unit.Nm()}],
        [{"value":3e2, "unit":N()}, {"value":-5e2, "unit":Nm()}],
        [{"value":4e1, "unit":Unit.N()}, {"value":-4e1, "unit":Unit.N()}],
        [{"value":1e3, "unit":Unit.N()}, {"value":1e3, "unit":Unit.N()}],
        [{"value":-6e1, "unit":Unit.N()}, {"value":5e2, "unit":Unit.N()}],
    ],
    allocations = [6] * 5
    ))
#q4
variables = [
    {"name":"i", "equation":"np.linspace(0,1,100)", "unit":non(), "first difine": 0},
    {"name":"y", "equation":"np.linspace(1,2,100)", "unit":Unit.m(), "first difine": 0},
    {"name":"x", "equation":"np.linspace(0, 2 / L , 10)", "unit" : Unit.m(), "first difine": 0},
    {"name":"EI", "value":2.5e5, "unit": Nm2(), "first difine": 0},
    {"name":"h", "value":20e-3, "unit": Unit.m(), "first difine": 0},
    {"name":"b", "value":30e-3, "unit": Unit.m(), "first difine": 0},
    {"name":"w0", "value":1e2, "unit": Unit.N_per_m(), "first difine": 0},
    {"name":"L", "value":1e1, "unit": Unit.m(), "first difine": 0},
    {"name":"L^2", "equation":" L ** 2", "unit": Unit.m2(), "first difine": 0},
    {"name":"L^3", "equation":" L ** 3", "unit": Unit.m3(), "first difine": 0},
    {"name":"L^4", "equation":" L ** 4", "unit": Unit.m4(), "first difine": 0}
]

corrects = [
    {"name":"RA", "equation":" w0 * L "},
    {"name":"RB", "equation":" w0 * L "},
    {"name":"a", "equation":" -1 * w0 * x / 2 / L "},
    {"name":"x1", "equation":" y "},
    {"name":"x2", "equation":" y / L "},
    {"name":"theta", "equation":" 5 * w0 * L ** 3 / 48 / EI "},
    {"name":"delta", "equation":" 13 * w0 * L ** 4 / 384 / EI "},    
    ]

question_classes = [
    Exam.EquationAndNumberQuestion,
    Exam.EquationAndNumberQuestion,
    Exam.EquationQuestion,
    Exam.EquationQuestion,
    Exam.EquationQuestion,
    Exam.EquationAndNumberQuestion,
    Exam.EquationAndNumberQuestion
    ]

exam.parts.append(
    Exam.Part(
        marks = np.vstack([exam.marks[0][7:37, 29:55], exam.marks[1][2:7, 1:27]]),
        variables = variables,
        corrects = corrects,
        allocations = [2] * 5 + [5] * 2,
        question_classes=question_classes,
        unit_instances=unit_instances
        )
    )


exam.scoring()

print(exam.score)
 

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.cidfonts import UnicodeCIDFont

from reportlab.platypus import Table, TableStyle
from reportlab.lib import colors
from reportlab.lib.units import mm   

def make_pdf(savepath, filename, scan_top, scan_back ,exam):

    """
    https://www.aligns.co.jp/blog/python/
    返却用採点結果pdfの作成

    corrects, answers, scoring: 問題毎の正答，回答，正誤のリスト
    score　問題毎の得点
    """

    pdf_canvas = canvas.Canvas(f"{savepath}/{filename}.pdf") 
    pdf_canvas.setAuthor("Yusaku MAEDA")
    pdf_canvas.setTitle(filename)
    pdf_canvas.setSubject("")


    pdfmetrics.registerFont(UnicodeCIDFont('HeiseiKakuGo-W5')) 
    
    from reportlab.lib.pagesizes import A4, portrait #用紙の向き
    width, height = A4 

    #日付
    pdf_canvas.setFillColorRGB(0, 0, 0)
    import datetime
    t_delta = datetime.timedelta(hours=9)
    JST = datetime.timezone(t_delta, "JST")
    now = datetime.datetime.now(JST)
    ymd = now.strftime("%Y/%m/%d %H:%M")
                   
    font_size = 4*mm #フォントサイズ
    pdf_canvas.setFont('HeiseiKakuGo-W5', font_size)
    pdf_canvas.drawString(145*mm, (287-5)*mm,  f"更新日時: {ymd}")    

    #タイトル
    pdf_canvas.setFillColorRGB(0, 0, 0); font_size = 6*mm; pdf_canvas.setFont('HeiseiKakuGo-W5', font_size)
    pdf_canvas.drawString(10*mm, (297-10-6-5)*mm, "材料力学Ⅱ　前期期末試験　採点結果") #書き出し(横位置, 縦位置, 文字)
    
    
    x = [0, 0.5, 0, 0.5, 0, 0, 0.33, 0.66]
    y = [0,   0, 1,   1, 2, 3,    3,    3]
    for p, part in enumerate(exam.parts):

        font_size = 4*mm
        pdf_canvas.setFillColorRGB(0, 0, 0)
        pdf_canvas.setFont('HeiseiKakuGo-W5', font_size)
        pdf_canvas.drawString((20+x[p]*170)*mm, (132-y[p]*32)*mm, f"問{p+1}： 点") #書き出し(横位置, 縦位置, 文字)

    ## 1枚目終了(改ページ)
    pdf_canvas.showPage()
    
    #画像を入れ込む
    pdf_canvas.drawImage(scan_top, 10*mm, (297-134.3-10)*mm, 190*mm, 134.3*mm)
    pdf_canvas.drawImage(scan_back, 10*mm, (10)*mm, 190*mm, 134.3*mm)

    #名前隠し
    pdf_canvas.setFillColorRGB(1, 1, 1)
    pdf_canvas.rect(0*mm, (297-10-10)*mm, width*0.5, 17*mm, stroke=False, fill=True)

    font_size = 5*mm
    pdf_canvas.setFillColorRGB(0, 0, 0)
    pdf_canvas.setFont('HeiseiKakuGo-W5', font_size)
    pdf_canvas.drawString(10*mm, (297-10-5)*mm, "答案とマーク読取結果") #書き出し(横位置, 縦位置, 文字)

    pdf_canvas.save()

make_pdf("./", "result", r".\files\result\image\top_sample.jpg", r".\files\result\image\back_sample.jpg", exam)


In [ ]:
exam.parts[3].questions[0].answers[0].variables

In [ ]:
for p in exam.parts:
    for q in p.questions:
        for a in q.answers:
            print(a)

In [ ]:
for p in exam.parts:
    print(p.score)

q4

In [ ]:
for p in exam.parts[3].questions:    
    print(p.score)

In [ ]:
exam.parts[3].questions[0].answers[0].score

In [ ]:
from packages import Marksheet

marker_positions = [
    ( 2, 26),
    ( 2, 28),
    ( 2, 56),
    ( 3, 24),
    (40,  2),
    (40, 56),
]

sheet = Marksheet(path = "./files/scan/back_sample.jpg", 
                  row = 41, column = 57, 
                  marker_positions = marker_positions,
                  binary_threshold=240,
                  monitor= True)
sheet.read()
sheet.rotation()
sheet.aliment()
sheet.threshold_check()
sheet.make_grids()
sheet.mark_check()
sheet.draw_grid()
sheet.save()

In [ ]:
from packages import Marksheet

marker_positions = [
    ( 2, 28),
    ( 2, 56),
    (40,  2),
    (40,  4),
    (40, 56),
]

sheet = Marksheet(path = "./files/scan/top_sample.jpg", 
                  row = 41, column = 57, 
                  marker_positions=marker_positions,
                  binary_threshold=240,
                  monitor= True)

sheet.read()
sheet.rotation()
sheet.aliment()
sheet.draw_grid()
sheet.threshold_check()
sheet.make_grids()
sheet.mark_check()
sheet.save()
